In [133]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

np.set_printoptions(suppress=True,)

In [134]:
header  = ["Click", "Weekday", "Hour", "Timestamp", "Log Type", "User ID", "User‐Agent", "IP", "Region", "City", "Ad Exchange", "Domain", "URL", "Anonymous URL ID", "Ad slot ID", "Ad slot width", "Ad slot height", "Ad slot visibility", "Ad slot format", "Ad slot floor price (RMB/CPM)", "Creative ID", "Key Page URL", "Advertiser ID", "User Tags"]
print(alist)

['Click', 'Weekday', 'Hour', 'Timestamp', 'Log Type', 'User ID', 'User\xe2\x80\x90Agent', 'IP', 'Region', 'City', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL ID', 'Ad slot ID', 'Ad slot width', 'Ad slot height', 'Ad slot visibility', 'Ad slot format', 'Ad slot floor price (RMB/CPM)', 'Creative ID', 'Key Page URL', 'Advertiser ID', 'User Tags']


In [135]:
trainfile = 'trainhead100000.txt'
testfile = 'testhead.txt'

train = pd.read_csv(trainfile, header = None, sep = '\t', names = header)
test = pd.read_csv(testfile, header = None, sep = '\t', names = header[1:])

### Show head lines of files
# print(train.head())
#print(test.head())

### output to csv file
# train.head().to_csv('out.csv', sep = '\t')

In [136]:
### Check the structure of input files
# print(train.describe())
# print(train.shape)

In [137]:
### Logistic Regression
mytrain = train.ix[:, [1,2,8,9,10,15,16,17,18]]
mytest = test.ix[:, [0,1,7,8,9,14,15,16,17]]
#print(mytest.describe())

X,x,Y,y = train_test_split(mytrain, train["Click"], test_size = 0.1)

lr = LogisticRegression(C = 1e+5)
lr.fit(X, Y)
print("The result of validation test is : %f" % lr.score(x,y))
### Predict class label for test samples
# result = lr.predict(mytest)

### Predict probability for each class label
#result = lr.predict_proba(mytest)

#print(result[:100, 1])

The result of validation test is : 0.999300
